In [1]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import ExperimentalMarkdownSyntaxTextSplitter
from langchain_text_splitters import  CharacterTextSplitter


headers_to_split_on = [
    ("#", "h1"),
    ("##", "h2"),
    ("###", "h3"),
    ("####", "h4"),
]
markdown_splitter = ExperimentalMarkdownSyntaxTextSplitter(
    headers_to_split_on=headers_to_split_on,
)
with open('MinerU_2307.09288v2__20251127030211.md', 'r') as file:
    content = file.read()
    md_header_splits = markdown_splitter.split_text(content)


In [ ]:
md_header_splits


In [ ]:
res = []
for doc in md_header_splits:
    metadata = doc.metadata
    headings = []
    if "h1" in metadata:
        headings.append(metadata["h1"])
    if "h2" in metadata:
        headings.append(metadata["h2"])
    if "h3" in metadata:
        headings.append(metadata["h3"])
    if "h4" in metadata:
        headings.append(metadata["h4"])

    res.append({
        "headings": headings,
        "content": doc.page_content
    })

In [3]:
import re

id_count = 1

heading_tree = dict()
for doc in md_header_splits:
    metadata = doc.metadata
    headings = []
    if "h1" in metadata:
        headings.append(metadata["h1"])
    if "h2" in metadata:
        headings.append(metadata["h2"])
    if "h3" in metadata:
        headings.append(metadata["h3"])
    if "h4" in metadata:
        headings.append(metadata["h4"])
    length = len(headings)
    def formatHeader(i: int) -> str :
        header = '#' * (i + 1)
        return f"{header} {headings[i]}"
    obj = heading_tree.setdefault(formatHeader(length - 1), {"child": []})
    obj["id"] = id_count
    id_count += 1
    obj["headings"] = headings
    obj["chunks"] = re.split(r'\n{2,}', doc.page_content.strip())
    for i in range(length - 1):
        parent = formatHeader(i)
        child = formatHeader(i + 1)
        obj = heading_tree.setdefault(parent, {"child": []})
        if child not in obj["child"] :
            obj["child"].append(child)

In [4]:
len(heading_tree)

65

In [ ]:
import json

with open("doc.json", 'w') as json_file:
    json.dump(res, json_file, indent=2)

In [54]:
for i in range(4):
    doc = md_header_splits[i]
    h = doc.metadata["headings"]
    for line in h:
        print(line)
    print(doc.page_content)
    value = heading_tree.get(h[-1], {"child": []})
    for line in value["child"]:
        print(line)

# LLAMA 2: Open Foundation and Fine-Tuned Chat Models
Hugo Touvron* Louis Martin† Kevin Stone†  
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Ed

"Identify the key concepts mentioned in the following document. For each key concept, 
summarize how it is introduced, explained, and discussed throughout the document. Be sure to highlight any examples, case studies, or specific arguments related to each concept."

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="anthropic/claude-haiku-4.5",  # Specify a model available on OpenRouter
    temperature=0.1,
    api_key="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",
    base_url="https://openrouter.ai/api/v1",
)

In [97]:
summary_prompt = """
You are an expert document summarizer. Your task is to summarize and refine document.

You MUST:
- Identify the main concepts of the document. 
- Briefly describe the main concepts, what it is.
- Summarize what the content introduce and discuss about the main concept.
- Summarize what each sub section introduce and discuss about the main concept.
- keep the summary simple, short and concise.

IMPORTANT: you MUST summarizze the document solely on the document contents.
IMPORTANT: DO NOT use markdown headings, output two paragraph, first identify and describe the main concept, second summarize how the main concept is discessed.
IMPORTANT: refer each sub section in this format: sub-section ** <name> **.

Response examples:
<example>
This section discuss about the main concept vector database, it is a database searching by vector, mainly used for semantic search.

sub-section ** feature ** discuss about the feature about the vector database.
sub-section ** usage ** discuss about the how to use vector database.
</example>

The contents under each sub section is omitted.

Summarize this document.

<DOCUMENT>
{}
</DOCUMENT>

"""

In [111]:
chunk_prompt = """
You are an expert document chunk summarizer. Your task is to summarize and refine each chunk in document.
The document contents is separate to several chunks.

You MUST:
- Identify the core concepts of each chunk, figure out what each chunk is talk about.
- Briefly describe the core concepts, what it is.
- For each chunk, summarize and refine what this chunk introduce and discuss about the core concept.
- keep the summary simple, short and concise.

IMPORTANT: for each chunk summary, do not use markdown headings, output two paragraph, first describe the core concepts, second describe how the core concept is discussed in this chunk.
IMPORTANT: refer each sub section in this format: sub-section ** <name> **.
IMPORTANT: for each chunk summary, do not involve contents out of this chunk.

Summarize each chunk in this document.

<DOCUMENT>
{}
</DOCUMENT>

"""

In [98]:

doc = """
# LLAMA 2: Open Foundation and Fine-Tuned Chat Models
Hugo Touvron* Louis Martin† Kevin Stone†  
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov Thomas Scialom*
## Abstract
## Contents
## 1 Introduction
## 2 Pretraining
## 3 Fine-tuning
## 4 Safety
## 5 Discussion
## 6 Related Work
## 7 Conclusion
## References
## A Appendix
"""

msg = summary_prompt.format(doc)
resp = model.invoke(msg)

In [114]:

chunk_doc = """

Section structure:
```
# LLAMA 2: Open Foundation and Fine-Tuned Chat Models

<chunk contents>

## Abstract
## Contents
## 1 Introduction
## 2 Pretraining
## 3 Fine-tuning
## 4 Safety
## 5 Discussion
## 6 Related Work
## 7 Conclusion
## References
## A Appendix
```

Chunk contents:
```
# LLAMA 2: Open Foundation and Fine-Tuned Chat Models

<chunk>
<id>1</id>
Hugo Touvron* Louis Martin† Kevin Stone†  
Peter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra
Prajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen
Guillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller
Cynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou
Hakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev
Punit Singh Koura Marie-Anne Lachaux Thibaut Lavril Jenya Lee Diana Liskovich
Yinghai Lu Yuning Mao Xavier Martinet Todor Mihaylov Pushkar Mishra
Igor Molybog Yixin Nie Andrew Poulton Jeremy Reizenstein Rashi Rungta Kalyan Saladi
Alan Schelten Ruan Silva Eric Michael Smith Ranjan Subramanian Xiaoqing Ellen Tan Binh Tang
Ross Taylor Adina Williams Jian Xiang Kuan Puxin Xu Zheng Yan Iliyan Zarov Yuchen Zhang
Angela Fan Melanie Kambadur Sharan Narang Aurelien Rodriguez Robert Stojnic
Sergey Edunov Thomas Scialom*
</chunk>
```

"""

msg = chunk_prompt.format(chunk_doc)
resp = model.invoke(msg)

In [115]:
print(resp.content)

# Chunk 1 Summary

The core concept of this chunk is the **authorship and attribution** of the LLAMA 2 research paper. This section identifies the large collaborative team of researchers responsible for developing and publishing the LLAMA 2 open foundation and fine-tuned chat models.

This chunk presents an extensive list of author names affiliated with the LLAMA 2 project, establishing the collaborative effort behind the research. The authors are listed with their names and institutional affiliations (indicated by symbols like * and †), representing a significant research team involved in creating the foundation models and chat variants discussed in the paper. This authorship section serves as the formal attribution for the work presented throughout the document.


In [121]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ProviderStrategy

class summary(BaseModel):
    id: int
    summary: str

class Response(BaseModel):
    chunk_summary: list[summary]


agent = create_agent(
    model=model,
    system_prompt=chunk_prompt.format(chunk_doc),
    response_format=ProviderStrategy(Response),
)

In [124]:
response = agent.invoke(
    input={},
)

StructuredOutputValidationError: Failed to parse structured output for tool 'Response': Native structured output expected valid JSON for Response, but parsing failed: Expecting value: line 1 column 1 (char 0)..